<a href="https://colab.research.google.com/github/iraidaantropova/data-processing-method/blob/main/%D0%9F%D0%A04_%D0%A1%D0%B8%D0%BD%D0%B8%D1%86%D0%B0%D0%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание

1.Развернуть у себя на компьютере/виртуальной машине MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу

2.* Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы (необходимо анализировать оба поля зарплаты)

3.* Любая аналитика. Например, matching вакансиий с разных площадок

In [1]:
!apt install mongodb > log

In [2]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [3]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from pymongo import MongoClient
from pymongo.errors import InvalidDocument as idoc
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint

In [10]:
client = MongoClient() # Подготовим базу данных

In [11]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [24]:
db = client.hr_db
vacations = db.hh_vacations


In [34]:
url = 'https://hh.ru/search/vacancy'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 YaBrowser/22.11.3.818 Yowser/2.5 Safari/537.36',    
}

In [35]:
def get_html(url, params=''):
    response = requests.get(url, headers=headers, params=params)
    return response

In [36]:
# response = get_html(url, params={'area': '113', 'text': 'python', 'page': 0})

def get_hh_content(response):
    soup = bs(response.text, 'html.parser')
    items = soup.find_all('div', class_='vacancy-serp-item')
    vacancy = []
    for item in items:
        title = item.find('span', attrs={'class': 'resume-search-item__name'})
        link = title.find('a').get('href')
        city = item.find('span', attrs={'class': 'vacancy-serp-item__meta-info'}).get_text()
        company = item.find('div', attrs={'class': 'vacancy-serp-item__meta-info-company'}).get_text()
        try:
            salary = item.find('div', attrs={'class': 'vacancy-serp-item__sidebar'}).get_text()
        except:
            salary = ''

        vacancy.append(
            {
                'title': title.get_text(),
                'link': link,
                'city': city,
                'company': company,
                'salary': salary,
                'site': 'HeadHunter'
             }
        )
    return vacancy

In [37]:
def edit_data_hh(vacancy):
    for vac in vacancy:
        # Del \xa0
        vac['company'] = vac['company'].replace(u'\xa0', u' ')
        vac['salary'] = vac['salary'].replace(u'\u202f', u'')

        # Salary (min, max, currency)
        if vac['salary']:
            salary_list = vac['salary'].split(' ')
            if salary_list[0] == 'от':
                vac['salary_min'] = float(salary_list[1])
                vac['salary_max'] = None
            elif salary_list[0] == 'до':
                vac['salary_min'] = None
                vac['salary_max'] = float(salary_list[1])
            else:
                vac['salary_min'] = float(salary_list[0])
                vac['salary_max'] = float(salary_list[2])
            vac['salary_currency'] = salary_list[-1]
        else:
            vac['salary_min'] = None
            vac['salary_max'] = None
            vac['salary_currency'] = None
        vac.pop('salary')

        # Id
        id = vac['link'].split(sep='/')[4].split(sep='?')[0]
        # id = id.split(sep='?')[0]
        vac['_Id'] = id
    return vacancy

def main():
    search_name = input('Название вакансии для поиска: ')
    page_count = int(input('Номер страницы для поиска '))

    result = []
    for i in range(page_count):
        print(f'Page {i+1}')
        response = get_html(url, params={'area': '113', 'text': search_name, 'page': i})
        vacancy = get_hh_content(response)
        vacancy_edit = edit_data_hh(vacancy)
        if response.ok and vacancy:
            result.extend(vacancy_edit)
        else:
            break

    return result

result = main()
# pprint(result)

Название вакансии для поиска: администратор
Номер страницы для поиска 1
Page 1


In [38]:
# Добавляем данные в монго
def insert_mongo(result):
    for vac in result:
        try:
            vacations.update_one({'_Id': vac['_Id']}, {'$set': vac}, upsert=True)
        except idoc:
            print(f'Error {idoc}')
            continue

insert_mongo(result)